In [16]:
#libraries
import numpy as np
import matplotlib.pyplot as plt
import math 

In [43]:
#input variables
n = int(input("Numero de usuarios: "))
proConnect =  float(input("Probabilidad que se conecte un usuario: "))
proDisconnect =  float(input("Probabilidad que se desconecte un usuario: "))

In [44]:
#variables
A = np.zeros((n,n)) #matriz de probabilidad

In [45]:
#functions
def combinatoria(n,x):
    if x > n or x < 0:
        return 0
    else:
        c = math.factorial(n)/ (math.factorial(x) * math.factorial(n-x ))
        return c
 

i hace referencia a la cantidad de usuarios conectados en ese momento
conn hace referencia al numero de usuarios que se pueden conectar
disc hace referencia al numero de usuarios que se pueden desconectar
n numero de usuarios que usan el ordenador

In [60]:
#code
for i in range(n):
    for j in range(n):
        #si no hay ningun usuario conectado probabilidad que se conecten usuarios
        if i == 0:
            conn = j
            A[i][j] = combinatoria(n,conn) * (proConnect ** conn) * ((1 - proConnect)**(n - conn))  

        #si todos los usuarios estan conectados probabilidad de desconectar usuarios
        elif i == n: 
            disc = i - j 
            A[i][j] = combinatoria(n, disc) * (proDisconnect ** disc) * ((1 - proDisconnect)**(n-disc)) 
        
        #si el usuario actual se desconecta entra  uno nuevo o no hay ningun cambio
        
        
        #no se desconecta ningun usuario y se conecta uno nuevo
        else:
            con = 1

        

In [62]:
#graphics
com = combinatoria(7,4)
print(n , " ** ",proConnect, " ** ",proDisconnect)
print(com)

suma = 0

for j in range(n):
    suma += A[0][j]
    
print(suma)
print(A)

10  **  0.5  **  0.2
35.0
0.9990234375
[[0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171875  0.20507812 0.24609375
  0.20507812 0.1171875  0.04394531 0.00976562]
 [0.00097656 0.00976562 0.04394531 0.1171